In [13]:
import torch
import numpy as np

In [14]:
"""
Init signature: torch.nn.Linear(in_features, out_features, bias=True)
Docstring:     
Applies a linear transformation to the incoming data: :math:`y = xA^T + b`

Args:
    in_features: size of each input sample
    out_features: size of each output sample
    bias: If set to ``False``, the layer will not learn an additive bias.
        Default: ``True``
"""
net = torch.nn.Linear(4, 2)

In [15]:
x = torch.arange(0, 4).float()
print(x)
# y = net.forward(x) # alternatively
y = net(x)
print(y)

tensor([0., 1., 2., 3.])
tensor([-0.7402,  0.0309], grad_fn=<AddBackward0>)


In [16]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.0015, -0.0476,  0.2135, -0.2137],
        [ 0.1825,  0.2287,  0.3416, -0.1389]], requires_grad=True)
Parameter containing:
tensor([-0.4785, -0.4641], requires_grad=True)


You can use small networks inside big networks. Parameters of subnetworks will be absorbed.
Here we're passing (def forward) parameters sequentially one by one through layers

In [17]:
class MyNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.layer1 = torch.nn.Linear(input_size, hidden_size)
        self.layer2 = torch.nn.Sigmoid()
        self.layer3 = torch.nn.Linear(hidden_size, output_size)
    
    def forward(self, input_val):
        h = input_val
        h = self.layer1(h)
        h = self.layer2(h)
        h = self.layer3(h)
        return h

net = MyNet(4, 16, 2)

This is a shortcut for simple feedforward networks. 

In [18]:
def generate_net(input_size, hidden_size, output_size):
    return torch.nn.Sequential(torch.nn.Linear(input_size, hidden_size),
                              torch.nn.ReLU(), 
                              torch.nn.Linear(hidden_size, output_size))

net = generate_net(4, 16, 2)

Final Layers and Losses

In [22]:
x = torch.tensor([np.arange(4), np.zeros(4), np.ones(4)]).float()
print(x)
y = torch.tensor([0, 1, 0])
print(y)
criterion = torch.nn.CrossEntropyLoss()
output = net(x)
print(output)
loss = criterion(output, y)
print(loss)

tensor([[0., 1., 2., 3.],
        [0., 0., 0., 0.],
        [1., 1., 1., 1.]])
tensor([0, 1, 0])
tensor([[ 0.1432, -0.2828],
        [ 0.0435, -0.0924],
        [ 0.0106,  0.0714]], grad_fn=<AddmmBackward>)
tensor(0.6634, grad_fn=<NllLossBackward>)


How to use the optimizer
-Remember that backpropagation in pytorch accumulates.
If you want to apply several iterations of gradient descent, gradients must be set to zero before each optimization step.

In [23]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
n_iter = 100
for i in range(n_iter):
    optimizer.zero_grad()
    output = net(x)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()